In [1]:
# Importar librerias
import pandas as pd
import pyarrow.parquet as pq

In [2]:
# Se establece la ruta del archivo Parquet
archivo2023 = 'C:/Users/User/Desktop/PF_NYC_CO2/Taxi/Taxi_2023.parquet'
# Se lee el archivo Parquet y se configura la  opcion de lectura para optimizar la memoria
tabla_parquet = pq.read_table(archivo2023, use_threads=True)
# Se convierte la tabla a un DataFrame de pandas y se libera la memoria utilizada por la tabla Parquet
df2023 = tabla_parquet.to_pandas()

In [3]:
# Se muestra el DataFrame
df2023.head()

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,airport_fee,Airport_fee
0,2,2023-01-01 00:32:10,2023-01-01 00:40:36,1.0,0.97,1.0,N,161,141,2,9.3,1.00,0.5,0.00,0.0,1.0,14.30,2.5,0.00,NaN
1,2,2023-01-01 00:55:08,2023-01-01 01:01:27,1.0,1.10,1.0,N,43,237,1,7.9,1.00,0.5,4.00,0.0,1.0,16.90,2.5,0.00,NaN
2,2,2023-01-01 00:25:04,2023-01-01 00:37:49,1.0,2.51,1.0,N,48,238,1,14.9,1.00,0.5,15.00,0.0,1.0,34.90,2.5,0.00,NaN
3,1,2023-01-01 00:03:48,2023-01-01 00:13:25,0.0,1.90,1.0,N,138,7,1,12.1,7.25,0.5,0.00,0.0,1.0,20.85,0.0,1.25,NaN
4,2,2023-01-01 00:10:29,2023-01-01 00:21:19,1.0,1.43,1.0,N,107,79,1,11.4,1.00,0.5,3.28,0.0,1.0,19.68,2.5,0.00,NaN


In [4]:
# Se eliminan las columnas que no seran utilizadas en el modelado
columnas_a_eliminar = ['VendorID', 'payment_type','tip_amount', 'tolls_amount','congestion_surcharge', 
                        'airport_fee', 'Airport_fee', 'extra', 'mta_tax','improvement_surcharge', 'fare_amount']
df2023 = df2023.drop(columns=columnas_a_eliminar)

In [5]:
df2023.shape

(34933659, 9)

In [6]:
# Se visualiza el contenido de la columna passenger_count
df2023['passenger_count'].value_counts()

passenger_count
1.0    25476904
2.0     5095843
3.0     1257595
4.0      696499
0.0      540555
5.0      443803
6.0      292773
8.0         208
7.0          76
9.0          50
Name: count, dtype: int64

In [10]:
# Se eliminan las filas donde passenger_count es igual a 0.0
df2023= df2023[df2023['passenger_count'] != 0.0]
df2023['passenger_count'].value_counts()

passenger_count
1.0    25476904
2.0     5095843
3.0     1257595
4.0      696499
5.0      443803
6.0      292773
8.0         208
7.0          76
9.0          50
Name: count, dtype: int64

In [12]:
# Se visualiza el contenido de la columna trip_distance
df2023['trip_distance'].value_counts()

trip_distance
0.00        686047
1.00        446668
0.90        446039
1.10        435638
0.80        431738
             ...  
146.00           1
125.30           1
54.64            1
104.40           1
63068.23         1
Name: count, Length: 8328, dtype: int64

In [13]:
# Eliminamos las filas donde trip_distance es igual a 0.00
df2023 = df2023[df2023['trip_distance'] != 0.00]
df2023['trip_distance'].value_counts()

trip_distance
1.00        446668
0.90        446039
1.10        435638
0.80        431738
1.20        423183
             ...  
146.00           1
125.30           1
54.64            1
104.40           1
63068.23         1
Name: count, Length: 8327, dtype: int64

In [15]:
# Se validan los valores nulos
df2023.isnull().sum()

tpep_pickup_datetime          0
tpep_dropoff_datetime         0
passenger_count          868279
trip_distance                 0
RatecodeID               868279
store_and_fwd_flag       868279
PULocationID                  0
DOLocationID                  0
total_amount                  0
dtype: int64

In [16]:
# Se eliminan los valores nulos
df2023 = df2023.dropna()

In [14]:
duplicados = df2023.duplicated()
duplicados

0           False
1           False
2           False
4           False
5           False
            ...  
34933653    False
34933654    False
34933656    False
34933657    False
34933658    False
Length: 33707057, dtype: bool

In [17]:
# Se obtiene información del dataframe
df2023.info()

<class 'pandas.core.frame.DataFrame'>
Index: 32838778 entries, 0 to 34800983
Data columns (total 9 columns):
 #   Column                 Dtype         
---  ------                 -----         
 0   tpep_pickup_datetime   datetime64[us]
 1   tpep_dropoff_datetime  datetime64[us]
 2   passenger_count        float64       
 3   trip_distance          float64       
 4   RatecodeID             float64       
 5   store_and_fwd_flag     object        
 6   PULocationID           int64         
 7   DOLocationID           int64         
 8   total_amount           float64       
dtypes: datetime64[us](2), float64(4), int64(2), object(1)
memory usage: 2.4+ GB


In [18]:
archivo_csv = "C:/Users/User/Desktop/PF_NYC_CO2/Datasets/taxi+_zone_lookup.csv"
# Se lee el archivo CSV en un DataFrame de pandas
df = pd.read_csv(archivo_csv)
df.head()

,LocationID,Borough,Zone,service_zone
0,1,EWR,Newark Airport,EWR
1,2,Queens,Jamaica Bay,Boro Zone
2,3,Bronx,Allerton/Pelham Gardens,Boro Zone
3,4,Manhattan,Alphabet City,Yellow Zone
4,5,Staten Island,Arden Heights,Boro Zone


In [19]:
df.isnull().sum()

LocationID      0
Borough         0
Zone            1
service_zone    2
dtype: int64

In [20]:
# Se renombra la columna LocationID	a PULocationID para unir los dataframes
df = df.rename(columns={'LocationID': 'PULocationID'})
df.head()

,PULocationID,Borough,Zone,service_zone
0,1,EWR,Newark Airport,EWR
1,2,Queens,Jamaica Bay,Boro Zone
2,3,Bronx,Allerton/Pelham Gardens,Boro Zone
3,4,Manhattan,Alphabet City,Yellow Zone
4,5,Staten Island,Arden Heights,Boro Zone


In [21]:
# Se unen los dos dataframe df2023 con df 
df_taxi_2023 = pd.merge(df2023, df, on='PULocationID', how='inner') 

In [22]:
# Se reorganizan las columnas para una mejor lectura.
reorden = ['PULocationID','Borough','Zone','service_zone','tpep_pickup_datetime','tpep_dropoff_datetime',
           'passenger_count','trip_distance','DOLocationID','RatecodeID','store_and_fwd_flag','total_amount']

df_taxi_2023 = df_taxi_2023[reorden]

In [23]:
# Se renombran las columnas con el objetivo de mejorar la claridad y la legibilidad del conjunto de datos.
df_taxi_2023.rename(columns={'service_zone': 'Service_Zone','tpep_pickup_datetime': 'Tpep_pickup_datetime', 
                             'tpep_dropoff_datetime':'Tpep_dropoff_datetime','passenger_count': 'Passenger_Count', 
                             'trip_distance': 'Trip_Distance', 'store_and_fwd_flag':'Store_and_fwd_flag', 
                             'total_amount': 'Total_Amount'}, inplace=True)

In [24]:
df_taxi_2023.shape

(32838778, 12)

In [25]:
# Separamos la columna 'Tpep_pickup_datetime' en 'Date_pickup' y 'Time_pickup'
df_taxi_2023['Date_pickup'] = pd.to_datetime(df_taxi_2023['Tpep_pickup_datetime']).dt.date
df_taxi_2023['Time_pickup'] = pd.to_datetime(df_taxi_2023['Tpep_pickup_datetime']).dt.time

In [26]:
# Separamos la columna 'Tpep_dropoff_datetime' en 'Date_dropoff' y 'Time_dropoff'
df_taxi_2023['Date_dropoff'] = pd.to_datetime(df_taxi_2023['Tpep_dropoff_datetime']).dt.date
df_taxi_2023['Time_dropoff'] = pd.to_datetime(df_taxi_2023['Tpep_dropoff_datetime']).dt.time

In [27]:
# Se elimina la columna Tpep_pickup_datetime
df_taxi_2023.drop(columns=['Tpep_pickup_datetime'], inplace=True)
df_taxi_2023.drop(columns=['Tpep_dropoff_datetime'], inplace=True)

In [28]:
df_taxi_2023.head(1)

,PULocationID,Borough,Zone,Service_Zone,Passenger_Count,Trip_Distance,DOLocationID,RatecodeID,Store_and_fwd_flag,Total_Amount,Date_pickup,Time_pickup,Date_dropoff,Time_dropoff
0,161,Manhattan,Midtown Center,Yellow Zone,1.0,0.97,141,1.0,N,14.3,2023-01-01,00:32:10,2023-01-01,00:40:36


In [29]:
df_taxi_2023.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32838778 entries, 0 to 32838777
Data columns (total 14 columns):
 #   Column              Dtype  
---  ------              -----  
 0   PULocationID        int64  
 1   Borough             object 
 2   Zone                object 
 3   Service_Zone        object 
 4   Passenger_Count     float64
 5   Trip_Distance       float64
 6   DOLocationID        int64  
 7   RatecodeID          float64
 8   Store_and_fwd_flag  object 
 9   Total_Amount        float64
 10  Date_pickup         object 
 11  Time_pickup         object 
 12  Date_dropoff        object 
 13  Time_dropoff        object 
dtypes: float64(4), int64(2), object(8)
memory usage: 3.4+ GB


In [31]:
# Se cambia el tipo de dato de las columnas Date_pickup, Date_dropoff, Time_pickup y Time_dropoff a datatime
df_taxi_2023['Date_pickup'] = pd.to_datetime(df_taxi_2023['Date_pickup'], format='%Y-%m-%d')

In [32]:
df_taxi_2023['Date_dropoff'] = pd.to_datetime(df_taxi_2023['Date_dropoff'], format='%Y-%m-%d')

In [33]:
df_taxi_2023['Time_pickup'] = pd.to_datetime(df_taxi_2023['Time_pickup'], format='%H:%M:%S')

In [34]:
df_taxi_2023['Time_dropoff'] = pd.to_datetime(df_taxi_2023['Time_dropoff'], format='%H:%M:%S')

In [35]:
# Se guarda el DataFrame resultante en un archivo Parquet
df_taxi_2023.to_parquet('Taxi_2023_Unificado.parquet', index=False)